In [1]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from functools import reduce
url='https://books.toscrape.com/'
response=requests.get(url)

In [2]:
response=requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')
lst = (soup.find('ul', attrs = {'class':''}).find_all('li'))

In [3]:
links = []
for n in range(len(lst)):
    link = soup.find('ul', attrs = {'class':''}).find_all('li')[n].find('a').get('href')
    domain = url
    full_link = domain+link
    links.append(full_link)


In [11]:
headings = ['Book_name','Price','Rate','Category']
rate_dict = {'One':1, 'Two' : 2 , 'Three' : 3 , 'Four' : 4 , 'Five' : 5}
for url in links:
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    for book in soup.find_all('article',attrs={'class':'product_pod'}):
        pricex=float(book.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
        titlex=book.find('h3').find('a').get('title').strip()
        ratex=rate_dict[book.find('p').get('class')[1]]
        categoryx=soup.find('div',attrs={'class':'page-header action'}).find('h1').get_text()
        connection=sqlite3.connect('library_2.db')
        Cursor=connection.cursor()
        cursor=connection.execute('insert into books (name,price,category,rate) values(?,?,?,?)',[titlex,pricex,categoryx,ratex])
        connection.commit()
        connection.close()
    while soup.find('li',attrs={'class':'next'}):
        new_page_url=reduce(lambda a,b:a+'/'+b,url.split('/')[:-1])+'/'+soup.find('li',attrs={'class':'next'}).find('a').get('href')
        soup=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
        for book in soup.find_all('article',attrs={'class':'product_pod'}):
            pricex=float(book.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
            titlex=book.find('h3').find('a').get('title').strip()
            ratex=rate_dict[book.find('p').get('class')[1]]
            categoryx=soup.find('div',attrs={'class':'page-header action'}).find('h1').get_text()
            connection=sqlite3.connect('library_2.db')
            Cursor=connection.cursor()
            cursor=connection.execute('insert into books(name,price,category,rate) values(?,?,?,?)',[titlex,pricex,categoryx,ratex])
            connection.commit()
            connection.close()